In [9]:
# import necessary packages
import skimage
from skimage import measure
import pandas as pd
import numpy as np
import os, sys
import argparse
import time
import cv2
import math
import tqdm.notebook as tqdm
from multiprocessing import Pool

sys.path.append('../')

# custom
from utils.GetFileNames import get_file_names

In [14]:
# specify input / output directories
seg_path = '../data/test/test_predictions_by_pod/'

# Get segmentations filenames
seg_names = get_file_names(seg_path, '.png')
print('Number of segmentations:', len(seg_names))

Number of segmentations: 125


In [11]:
# Define function to extract features from segmentations
def extract_features(img, seg_path):

    # print action message
    print('Processing ' + img)
    # import segmentation
    segm = skimage.io.imread(seg_path + img)

    # if segm has 1 channels,
    if len(segm.shape) != 3:
        # save segmentation as gray image
        gray_image = segm
    else:
        # Convert the RGB image to grayscale
        gray_image = skimage.color.rgb2gray(segm)

    # Threshold the grayscale image to create a binary image
    binary_image = (gray_image > 0.5) * 255

    ######## Rotated rectangle ########

    # Find the contours of the binary mask
    contours = measure.find_contours(binary_image, 0.5)

    # Change the data type to float32
    contours = contours[0].astype(np.float32)

    # Calculate the minimum bounding rectangle
    rect = cv2.minAreaRect(contours)

    # get box points
    box = cv2.boxPoints(rect)

    # Create function to calculate width, height, hypotenuse and area of a rectangle
    def rect_prop(box_coords):
        '''
        Function to calculate width, height, hypotenuse and area of a rectangle
        '''
        # start empty list
        prop = []

        # extract euclidean distances between points
        for i in range(1,4):
            # extract x and y coordinates
            x = box[:,0]
            y = box[:,1]

            # compute euclidean distance
            dist = np.sqrt((x[0]-x[i])**2 + (y[0]-y[i])**2)

            # append to list
            prop.append(dist)

        # sort list
        prop.sort()

        # compute width, height, hypotenuse and area
        width = prop[0]
        height = prop[1]
        hypotenuse = prop[2]
        area = width*height

        # create dictionary compatible with pandas
        prop = {'rr_width': width, 'rr_height': height, 'rr_hypotenuse': hypotenuse, 'rr_area': area}

        # return dictionary
        return prop

    # apply function to extract features from rotated rectangle
    rot_rect = rect_prop(box)

    ######## Extract extra features using regions prop ########

    # Create feature list
    feature_list = [
    'area', 'bbox', 'area_bbox', 'eccentricity', 'equivalent_diameter_area',
    'euler_number', 'extent', 'feret_diameter_max', 'axis_major_length', 'axis_minor_length',
    'orientation', 'perimeter', 'perimeter_crofton', 'solidity', 'area_convex',
    'axis_major_length', 'axis_minor_length']
    
    # Extract mask features
    features = measure.regionprops_table(binary_image, properties=(feature_list))

    ######## Start dictionary and convert to data frame ########
    
    # convert segmentation name to dictionary
    features_dic = {"image_name" : img}
    
    # append features_dic at the beggining of features dictionary
    features_dic.update(features)

    # append rot_rect dictionary to features_dic
    features_dic.update(rot_rect)
    
    # Convert to dataframe
    features_df = pd.DataFrame(features_dic)

    ######## Compute extra features ########
    
    # Compute extra features
    features_df = (features_df
    # Width
     .assign(bb_width = features_df['bbox-3'] - features_df['bbox-1'])
    # Height
     .assign(bb_height = features_df['bbox-2'] - features_df['bbox-0'])
    # Circularity
     .assign(circularity = (4*np.pi*features_df['area'])/(features_df['perimeter']**2))
    # Aspect ratio
     .assign(aspect_ratio = features_df['axis_major_length']/features_df['axis_minor_length'])
    # Roundness
     .assign(roundness = (4*features_df['area'])/(np.pi*features_df['axis_major_length']**2))
    # rms
     .assign(rms = 2*(math.sqrt((0.5*features_df['axis_major_length'])**2-(0.5*features_df['axis_minor_length'])**2))/features_df['axis_major_length'])
     .drop(columns=['bbox-0', 'bbox-1', 'bbox-2', 'bbox-3']))
    
    # Reorder columns
    features_df = features_df[['image_name', 'area', 'area_bbox', 'area_convex', 
                               'bb_width', 'bb_height', 'rr_area', 'rr_width', 'rr_height', 
                               'rr_hypotenuse', 'perimeter', 'perimeter_crofton', 'equivalent_diameter_area', 
                               'euler_number', 'feret_diameter_max', 'axis_major_length', 'axis_minor_length', 
                               'orientation', 'aspect_ratio', 'circularity', 'eccentricity', 
                               'extent', 'rms', 'roundness', 'solidity']]
    
    # return features dataframe
    return features_df

In [16]:
for i, img in enumerate(seg_names[:3]):
    if i == 0:
        features_df = extract_features(img, seg_path)
    else:
        features_df = pd.concat([features_df, extract_features(img, seg_path)])


Processing pred_74.ISU367_A.B.ISU Wild.W_6.png
Processing pred_8.tt8t x fc6-8.G.WBP-2022.W_0.png
Processing pred_8.tt8t x fc6-8.G.WBP-2022.W_7.png


/tmp/ipykernel_342997/1232272121.py:113: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  .assign(rms = 2*(math.sqrt((0.5*features_df['axis_major_length'])**2-(0.5*features_df['axis_minor_length'])**2))/features_df['axis_major_length'])
/tmp/ipykernel_342997/1232272121.py:113: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  .assign(rms = 2*(math.sqrt((0.5*features_df['axis_major_length'])**2-(0.5*features_df['axis_minor_length'])**2))/features_df['axis_major_length'])
/tmp/ipykernel_342997/1232272121.py:113: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  .assign(rms = 2*(math.sqrt((0.5*features_df['axis_major_length'])**2-(0.5*features_df['axis_minor_length'])**2))/features_df['axis_major_length'])


In [17]:
features_df

,image_name,area,area_bbox,area_convex,bb_width,bb_height,rr_area,rr_width,rr_height,rr_hypotenuse,...,axis_major_length,axis_minor_length,orientation,aspect_ratio,circularity,eccentricity,extent,rms,roundness,solidity
0,pred_74.ISU367_A.B.ISU Wild.W_6.png,15230.0,29200.0,21497.0,146,200,28871.955954,164.269837,175.759326,240.574124,...,204.287659,151.092016,0.186312,1.352074,0.095376,0.673042,0.521575,0.673042,0.464650,0.708471
0,pred_8.tt8t x fc6-8.G.WBP-2022.W_0.png,18237.0,34398.0,24689.0,182,189,31948.186354,149.907054,213.119966,260.561412,...,218.024991,155.403131,-0.750024,1.402964,0.107394,0.701391,0.530176,0.701391,0.488485,0.738669
0,pred_8.tt8t x fc6-8.G.WBP-2022.W_7.png,14019.0,23056.0,17177.0,176,131,23043.355885,128.068535,179.929879,220.853594,...,184.624855,130.615558,-1.299235,1.413498,0.201603,0.706749,0.608041,0.706749,0.523657,0.816150
